In [3]:
!pip install docker
!pip install clickhouse_driver
!docker run -d --name clickhouse-server --ulimit nofile=262144:262144 -p 9000:9000 yandex/clickhouse-server

/bin/bash: line 1: docker: command not found


In [101]:
from clickhouse_driver import Client
import json
import pandas as pd

In [102]:
client = Client(host='huap0exaim.eu-central-1.aws.clickhouse.cloud',  # Use 'localhost' or '127.0.0.1' for a local server
                port=9440,
                user='default',    # Default user, adjust if you've changed the user
                password='wT0O~6xiFl~Hw',       # Default installation has no password for 'default' user
                secure=True)         # Default TCP port for ClickHouse

# Attempt to execute a query
try:
    result = client.execute('SHOW TABLES')
    print(result)
except Exception as e:
    print(f"Encountered an error: {e}")

[('trips',)]


In [105]:
client.execute('SHOW DATABASES')

[('INFORMATION_SCHEMA',), ('default',), ('information_schema',), ('system',)]

In [106]:
client.execute(
    '''
    CREATE TABLE books (
    id UInt64,
    Category String,
    Name String,
    Price String,
    In_stock String,
    In_stock_available UInt8,
    Link String
) ENGINE = MergeTree()
ORDER BY id;
    '''
)
try:
    result = client.execute('SHOW TABLES')
    print(result)
except Exception as e:
    print(f"Encountered an error: {e}")

[('books',), ('trips',)]


Читаем файл books_library.json. Список книг.

In [107]:
with open('/content/books_library.json', 'r') as infile:
    data = json.load(infile)  # Читаем массив объектов

In [108]:
# Преобразование данных в формат ClickHouse
rows = [(idx, item['Category'], item['Name'], item['Price'], item['In_stock'], item['In_stock_available'], item['Link']) for idx, item in enumerate(data, start=1)]

In [109]:
client.execute('INSERT INTO books (id, Category, Name, Price, In_stock, In_stock_available, Link) VALUES', rows)

11

In [110]:
data_from_books=client.execute('SELECT * FROM books')

In [111]:
df=pd.DataFrame(data_from_books, columns=("id", "Category", "Name", "Price", "In_stock", "In_stock_available", "Link")).drop(columns=['id'])

In [112]:
df

,Category,Name,Price,In_stock,In_stock_available,Link
0,Travel,It's Only the Himalayas,45.17,In stock (19 available),19,http://books.toscrape.com/catalogue//its-only-...
1,Travel,Full Moon over Noah’s Ark: An Odyssey to Mount...,49.43,In stock (15 available),15,http://books.toscrape.com/catalogue//full-moon...
2,Travel,See America: A Celebration of Our National Par...,48.87,In stock (14 available),14,http://books.toscrape.com/catalogue//see-ameri...
3,Travel,Vagabonding: An Uncommon Guide to the Art of L...,36.94,In stock (8 available),8,http://books.toscrape.com/catalogue//vagabondi...
4,Travel,Under the Tuscan Sun,37.33,In stock (7 available),7,http://books.toscrape.com/catalogue//under-the...
5,Travel,A Summer In Europe,44.34,In stock (7 available),7,http://books.toscrape.com/catalogue//a-summer-...
6,Travel,The Great Railway Bazaar,30.54,In stock (6 available),6,http://books.toscrape.com/catalogue//the-great...
7,Travel,A Year in Provence (Provence #1),56.88,In stock (6 available),6,http://books.toscrape.com/catalogue//a-year-in...
8,Travel,The Road to Little Dribbling: Adventures of an...,23.21,In stock (3 available),3,http://books.toscrape.com/catalogue//the-road-...
9,Travel,Neither Here nor There: Travels in Europe,38.95,In stock (3 available),3,http://books.toscrape.com/catalogue//neither-h...
